# Upload of evaluators
In this notebook we are demonstrating the upload of the standard evaluators.

### Import

In [ ]:
import os
import json
import pandas as pd
import shutil
import uuid
import yaml

from azure.ai.ml._ml_client import MLClient
from azure.identity._credentials.default import AzureCliCredential
from azure.ai.ml.entities import (
    Model
)

from promptflow.client import PFClient
from promptflow.evals.evaluate import evaluate
from promptflow.evals.evaluators import F1ScoreEvaluator

## End to end demonstration of evaluator sving and uploading to Azure.
### Saving the standard evaluators to the flex format.
First we will create the promptflow client, which will be used to save the existing flows.

In [ ]:
pf = PFClient()

We will use F1 score evaluator from the standard evaluator set and save it to local directory. 

In [ ]:
pf.flows.save(F1ScoreEvaluator, path='./f1_score')

Let us inspect, what has been saved

In [ ]:
print('\n'.join(os.listdir('f1_score')))

The file, defining entrypoint of our model is called flow.flex.yaml, let us display it.

In [ ]:
with open(os.path.join('f1_score', 'flow.flex.yaml')) as fp:
    flex_definition = yaml.safe_load(fp)
print(f"The evaluator entrypoint is {flex_definition['entry']}")

In [ ]:
pf = PFClient()
run = Run(
    flow='f1_score',
    data='data.jsonl',
    name=f'test_{uuid.uuid1()}'
)
run = pf.runs.create_or_update(
     run=run,
)
pf.stream(run)

<font color='red'>**Hack for standard evaluators.**</font> If we will try to load our evaluator directly, we will get an error, because we try to modify `__path__` variable, which will work inside the python package, however, it will fail if we will try to run evaluator as a standalone script. To fix it, we will remove this line from our code. **This code is not needed for custom user evaluators!**

In [ ]:
def fix_evaluator_code(flex_dir: str) -> None:
    """
    Read the flow.flex.yaml file from the flex_dir and remove the line, modifying __path__.

    :param flex_dir: The directory with evaluator saved in flex format.
    """
    with open(os.path.join(flex_dir, 'flow.flex.yaml')) as fp:
        flex_definition = yaml.safe_load(fp)
    entry_script = flex_definition['entry'].split(':')[0] + '.py'
    if entry_script == '__init__.py':
        with open(os.path.join(flex_dir, entry_script)) as f:
            with open(os.path.join(flex_dir, '_' + entry_script), 'w') as new_f:
                for line in f:
                    if not '__path__' in line:
                        new_f.write(line)
        os.replace(os.path.join(flex_dir, '_' + entry_script), os.path.join(flex_dir, entry_script))

fix_evaluator_code('f1_score')

Now let us test the flow with the simple dataset, consisting of one ground true and one actual sentense.

In [ ]:
data = pd.DataFrame({
    "ground_truth": ["January is the coldest winter month."],
    "answer": ["June is the coldest sommer month."]
})
in_file = 'data.jsonl'
data.to_json('data.jsonl', orient='records', lines=True, index=False)

Load the evaluator in a FLEX format and test it.

In [ ]:
flow_result = pf.test(flow='f1_score', inputs='data.jsonl')
print(f"Flow outputs: {flow_result}")

Now we have all the tools to upload our model to Azure
### Uploading data to Azure
First we will need to authenticate to azure. For this purpose we will use the the configuration file of the net structure.
```json
{
    "resource_group_nameup": "resource-group-name",
    "workspace_name": "ws-name",
    "subscription_id": "subscription-uuid",
    "registry_name": "registry-name"
}
```"


``````

In [ ]:
with open('config.json') as f:
    configuration = json.load(f)

#### Uploading to the workspace
In this scenario we will not need the `registry_name` in our configuration.

In [ ]:
config_ws = configuration.copy()
del config_ws["registry_name"]

credential = AzureCliCredential()
ml_client = MLClient(
    credential=credential,
    **config_ws
)

We will use the evauator operations API to upload our model to workspace.

In [ ]:
eval = Model(
    path="f1_score",
    name='f1_score_uploaded',
    description="F1 score evaluator.",
)
ml_client.evaluators.create_or_update(eval)

Now we will retrieve model and check that it is functional.

In [ ]:
ml_client.evaluators.download('f1_score_uploaded', version='1', download_path='f1_score_downloaed')

In [ ]:
flow_result = pf.test(flow=os.path.join('f1_score_downloaed', 'f1_score_uploaded', 'f1_score'), inputs='data.jsonl')
print(f"Flow outputs: {flow_result}")

In [ ]:
shutil.rmtree('f1_score_downloaed')
assert not os.path.isdir('f1_score_downloaed')

#### Uploading to the registry
In this scenario we will not need the `workspace_name` in our configuration.

In [ ]:
config_reg = configuration.copy()
del config_reg["workspace_name"]

ml_client = MLClient(
    credential=credential,
    **config_reg
)

We are creating new eval here, because create_or_update changes the model inplace, adding non existing link to workspace

In [ ]:
eval = Model(
    path="f1_score",
    name='f1_score_uploaded',
    description="F1 score evaluator.",
)
ml_client.evaluators.create_or_update(eval)

Now we will perfrm the same sanity check, we have done for the workspace.

In [ ]:
ml_client.evaluators.download('f1_score_uploaded', version='1', download_path='f1_score_downloaed')
flow_result = pf.test(flow=os.path.join('f1_score_downloaed', 'f1_score_uploaded', 'f1_score'), inputs='data.jsonl')
print(f"Flow outputs: {flow_result}")

Finally, we will do the cleanup.

In [ ]:
shutil.rmtree('f1_score_downloaed')
assert not os.path.isdir('f1_score_downloaed')

## Upload standard evaluators to the `mafong-registry`
Authenticate to registry client

In [ ]:
ml_client = MLClient(
    credential=credential,
    resource_group_name="mafong",
    subscription_id="2d385bf4-0756-4a76-aa95-28bf9ed3b625",
    registry_name="mafong-registry"
)

Take evaluators from two different namespaces: `evaluators` and `evaluators.content_safety`.

In [ ]:
import inspect
import tempfile

from promptflow.evals import evaluators
from promptflow.evals.evaluators import content_safety

def upload_models(namespace):
    """
    Upload all the evaluators in namespace.

    :param namespace: The namespace to take evaluators from.
    """
    for name, obj in inspect.getmembers(namespace):
        if inspect.isclass(obj):
            try:
                description = inspect.getdoc(obj) or inspect.getdoc(getattr(obj, '__init__'))
                with tempfile.TemporaryDirectory() as d:
                    os.makedirs(name, exist_ok=True)
                    artifact_dir = os.path.join(d, name)
                    pf.flows.save(obj, path=artifact_dir)
                    default_display_file = os.path.join(name, 'flow', 'prompt.jinja2')
                    properties = None
                    if os.path.isfile(os.path.join(d, default_display_file)):
                        properties = {'_default-display-file': default_display_file}
                    eval = Model(
                        path=artifact_dir,
                        name=name,
                        description=description,
                        properties=properties
                    )
                    #if not list(ml_client.evaluators.list(eval.name)):
                    ml_client.evaluators.create_or_update(eval)
                print(f'{name} saved')
            except BaseException as e:
                print(f'Failed to save {name} Error: {e}')


upload_models(evaluators)
upload_models(content_safety)